## 使用可变长度编码压缩倒排索引表

In [ ]:
# -*- coding: utf-8 -*-
# Author: Zhenyu Bo
# Date: 2024-11-07

"""
压缩倒排索引表并拆分为词典和倒排表文件。
"""

import json
import csv


def variable_byte_encode(gaps: list[int]) -> bytes:
    """
    使用可变字节编码（Variable Byte Encoding）对文档间距进行压缩。

    参数：
    - gaps: 文档间距列表。

    返回：
    - 压缩后的字节序列。
    """
    encoded_bytes = []
    for gap in gaps:
        byte_chunks = []
        while gap >= 128:
            byte_chunks.append(gap % 128)
            gap = gap // 128
        byte_chunks.append(gap + 128)  # 设置最后一个字节的最高位为1
        # 由于要按大端顺序存储，反转字节顺序
        byte_chunks = byte_chunks[::-1]
        encoded_bytes.extend(byte_chunks)
    return bytes(encoded_bytes)

def calculate_gaps(index_list: list[int]) -> list[int]:
    """
    计算文档间距列表。

    假设输入的 `index_list` 是一个递增的文档 ID 列表，返回每个文档 ID 与前一个文档 ID 之间的差值列表。

    参数：
    - index_list: 递增的文档 ID 列表。

    返回：
    - 文档间距列表。
    """
    gaps = []
    previous_id = 0  # 假设第一个文档的前一个 ID 为 0

    for doc_id in index_list:
        gap = doc_id - previous_id
        gaps.append(gap)
        previous_id = doc_id

    return gaps


def compress(index_list: list[int]) -> bytes:
    """
    压缩文档 ID 列表为字节序列。

    使用文档间距替代 ID 并应用可变字节编码进行压缩。

    参数：
    - index_list: 递增的文档 ID 列表。

    返回：
    - 压缩后的字节序列。
    """
    # 计算文档间距
    gaps = calculate_gaps(index_list)

    # 使用可变字节编码压缩间距列表
    compressed_data = variable_byte_encode(gaps)

    return compressed_data


def save_compressed_data(csv_file_path: str, compressed_file_path: str, vocabulary_file_path: str) -> None:
    """
    保存压缩后的倒排索引和词汇表。

    参数：
    - csv_file_path: 生成的倒排索引 CSV 文件路径（例如 "book_inverted_index_table.csv"）。
    - compressed_file_path: 压缩后的倒排表二进制文件路径。
    - vocabulary_file_path: 词汇表文件路径，包含词项及其在倒排表中的字节偏移量和长度。
    """
    # vocabulary = []
    current_offset = 0

    try:
        with open(compressed_file_path, "wb") as f_bin, \
             open(vocabulary_file_path, "w", encoding="UTF-8", newline='') as f_vocab, \
             open(csv_file_path, "r", encoding="UTF-8") as f_csv:

            csv_reader = csv.DictReader(f_csv)
            vocab_writer = csv.writer(f_vocab)
            # 写入词汇表的表头
            vocab_writer.writerow(['word', 'offset', 'length'])

            for row in csv_reader:
                word = row['word']
                # 解析 id_list（假设以列表字符串形式存储）
                id_list_str = row['id_list']
                id_list = json.loads(id_list_str)

                # 压缩文档 ID 列表
                compressed_bytes = compress(id_list)

                # 写入倒排表
                f_bin.write(compressed_bytes)

                # 记录词典信息
                vocab_writer.writerow([word, current_offset, len(compressed_bytes)])
                current_offset += len(compressed_bytes)

    except IOError as e:
        print(f"文件操作失败: {e}")
    except json.JSONDecodeError as e:
        print(f"JSON 解码失败: {e}")



# 处理书籍倒排索引表
csv_file_path = "../book_inverted_index_table.csv"
compressed_file_path = "../data/book_inverted_index_compressed.bin"
vocabulary_file_path = "../data/book_vocabulary.csv"

# 保存压缩后的倒排索引和词汇表
save_compressed_data(
    csv_file_path,
    compressed_file_path,
    vocabulary_file_path
)

# 处理电影倒排索引表
csv_file_path = "../data/movie_inverted_index_table.csv"
compressed_file_path = "../data/movie_inverted_index_compressed.bin"
vocabulary_file_path = "../data/movie_vocabulary.csv"

save_compressed_data(
    csv_file_path,
    compressed_file_path,
    vocabulary_file_path
)

## 实现在压缩的倒排索引表上查询一个词项的倒排列表的函数

In [ ]:
# -*- coding: utf-8 -*-
# Author: Zhenyu Bo
# Date: 2024-11-07

"""
根据词典文件查找并解压缩特定词项的倒排索引。
"""

import csv
import os


def variable_byte_decode(byte_data: bytes) -> list[int]:
    """
    使用可变字节解码（Variable Byte Decoding）将字节序列解码为整数列表。

    参数：
    - byte_data: 压缩的字节序列。

    返回：
    - 解码后的整数列表。
    """
    numbers = []
    current_number = 0
    for byte in byte_data:
        if byte >= 128:
            # 当最高位为1时，这是一个数字的第一个字节
            # 先将之前的数字加入列表
            if current_number != 0:
                numbers.append(current_number)
                current_number = 0
            # 去掉最高位的1，得到数字的高位
            current_number = (current_number << 7) | (byte - 128)
        else:
            # 当最高位为0时，这是一个数字的中间字节
            current_number = (current_number << 7) | byte
    # 将最后一个数字加入列表
    numbers.append(current_number)
    return numbers


def reconstruct_doc_ids(gaps: list[int]) -> list[int]:
    """
    根据文档间距列表重建原始的文档 ID 列表。

    参数：
    - gaps: 文档间距列表。

    返回：
    - 原始的文档 ID 列表。
    """
    doc_ids = []
    current_id = 0
    for gap in gaps:
        current_id += gap
        doc_ids.append(current_id)
    return doc_ids


def load_vocabulary(vocabulary_file_path: str) -> dict:
    """
    加载词典文件，将其内容存储在一个字典中。

    参数：
    - vocabulary_file_path: 词汇表文件路径，包含词项及其在倒排表中的字节偏移量和长度。

    返回：
    - 词典字典，键为词项，值为包含偏移量和长度的字典。
    """
    vocabulary = {}
    try:
        with open(vocabulary_file_path, "r", encoding="UTF-8") as f_vocab:
            csv_reader = csv.DictReader(f_vocab)
            for row in csv_reader:
                word = row['word']
                offset = int(row['offset'])
                length = int(row['length'])
                vocabulary[word] = {'offset': offset, 'length': length}
    except IOError as e:
        print(f"文件操作失败: {e}")
    except ValueError as e:
        print(f"数据解析失败: {e}")
    return vocabulary


def get_inverted_index_list(word: str, compressed_file_path="../data/book_inverted_index_compressed.bin",
                            vocabulary_file_path="../data/book_vocabulary.csv") -> list[int]:
    """
    根据给定的词项，从压缩的倒排表文件中提取并解压缩对应的倒排索引。

    参数：
    - word: 目标词项。
    - compressed_file_path: 压缩后的倒排表二进制文件路径。
    - vocabulary: 词典字典，包含词项及其在倒排表中的字节偏移量和长度。

    返回：
    - 解压缩后的文档 ID 列表。如果词项不存在，则返回空列表。
    """
    # 检查文件是否存在
    if not os.path.exists(compressed_file_path):
        print(f"压缩文件 '{compressed_file_path}' 不存在。")
        return
    if not os.path.exists(vocabulary_file_path):
        print(f"词典文件 '{vocabulary_file_path}' 不存在。")
        return

    # 加载词典
    vocabulary = load_vocabulary(vocabulary_file_path)

    if word not in vocabulary:
        print(f"词项 '{word}' 不存在于词典中。")
        return []

    offset = vocabulary[word]['offset']
    length = vocabulary[word]['length']

    try:
        with open(compressed_file_path, "rb") as f_bin:
            f_bin.seek(offset)
            compressed_bytes = f_bin.read(length)
            gaps = variable_byte_decode(compressed_bytes)
            doc_ids = reconstruct_doc_ids(gaps)
            return doc_ids
    except IOError as e:
        print(f"文件操作失败: {e}")
        return []
    except (Exception) as e:
        print(f"解压缩过程中发生错误: {e}")
        return []

In [ ]:

"""
根据用户输入的词项，解压缩并显示其倒排索引。
"""
# 定义文件路径
compressed_file_path = "../data/book_inverted_index_compressed.bin"
vocabulary_file_path = "../data/book_vocabulary.csv"

while True:
    # 用户输入词项
    word = input("请输入要查询的词项（输入 'exit' 退出）：").strip()

    # 检查是否退出
    if word.lower() == 'exit':
        print("退出查询。")
        break

    # 获取并解压缩倒排索引
    doc_ids = get_inverted_index_list(word, compressed_file_path, vocabulary_file_path)

    if doc_ids:
        print(f"词项 '{word}' 对应的文档 ID 列表: {doc_ids}")
    else:
        print(f"词项 '{word}' 没有对应的文档或解压缩失败。")


## 使用压缩后的倒排索引表进行布尔查询

In [ ]:
# -*- coding: utf-8 -*-
# Author: Zhenyu Bo
# Date: 2024-11-14

"""
在经过可变长度编码压缩后的倒排索引表上执行布尔查询。
"""

import pandas as pd
import ast
import re

# 倒排索引表，movie_inverted_index_table.csv 
# 全id表，Movie_id.txt
# 词表，movie_words.csv 用于打印结果 目前格式同助教提供的selected_book_top_1200.csv

def read_all_ids(file_path):
    """
    读取所有ID，返回ID的集合。
    """
    all_ids = set()
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            id = int(line.strip())
            all_ids.add(id)
    return all_ids

def tokenize(expression):
    """
    将布尔表达式分词。
    """
    tokens = re.findall(r'AND|OR|NOT|\w+|\(|\)', expression)
    return tokens

def infix_to_postfix(tokens):
    """
    将中缀表达式转换为后缀表达式（逆波兰表达式）。
    """
    precedence = {'NOT': 3, 'AND': 2, 'OR': 1}
    output = []
    operator_stack = []
    operators = {'AND', 'OR', 'NOT'}
    for token in tokens:
        if token not in operators and token not in {'(', ')'}:
            output.append(token)
        elif token == '(':
            operator_stack.append(token)
        elif token == ')':
            while operator_stack and operator_stack[-1] != '(':
                output.append(operator_stack.pop())
            operator_stack.pop()  # 弹出 '('
        else:  # 操作符
            while operator_stack and operator_stack[-1] != '(' and precedence.get(operator_stack[-1], 0) >= precedence.get(token, 0):
                output.append(operator_stack.pop())
            operator_stack.append(token)
    while operator_stack:
        output.append(operator_stack.pop())
    return output

def evaluate_postfix(postfix_tokens, compressed_file_path, vocabulary_file_path, all_ids):
    """
    评估后缀表达式，返回符合条件的ID集合。
    """
    stack = []
    operators = {'AND', 'OR', 'NOT'}
    for token in postfix_tokens:
        if token not in operators:
            inverted_index = get_inverted_index_list(token, compressed_file_path, vocabulary_file_path)
            stack.append(set(inverted_index))
        elif token == 'NOT' or token == 'not':
            operand = stack.pop()
            stack.append(all_ids - operand)
        else:
            right = stack.pop()
            left = stack.pop()
            if token == 'AND' or token == 'and':
                stack.append(left & right)
            elif token == 'OR' or token == 'or':
                stack.append(left | right)
    return stack.pop() if stack else set()

def display_results(result_ids, words_df):
    """
    根据查询结果的ID，从 DataFrame 中查找并打印ID和标签。
    """
    df = words_df[words_df['id'].isin(result_ids)]
    if not df.empty:
        for _, row in df.iterrows():
            id = row['id']
            words = ast.literal_eval(row['words'])
            print(f"ID: {id}")
            print(f"标签: {', '.join(words)}\n")
    else:
        print("没有符合条件的结果。")


import time

# 主程序
print("正在加载数据，请稍候...")

# 加载书籍数据
book_all_ids = read_all_ids('../data/Book_id.txt')
book_words_df = pd.read_csv('../data/book_words.csv', dtype={'id': int, 'words': str})

# 加载电影数据
movie_all_ids = read_all_ids('../data/Movie_id.txt')
movie_words_df = pd.read_csv('../data/movie_words.csv', dtype={'id': int, 'words': str})

print("数据加载完成！")

while True:
    choice = input("请选择查询类型（1 - 书籍，2 - 电影）：\n")
    if choice == '1':
        compressed_file_path = "../data/book_inverted_index_compressed.bin"
        vocabulary_file_path = "../data/book_vocabulary.csv"
        all_ids = book_all_ids
        words_df = book_words_df
    elif choice == '2':
        compressed_file_path = "../data/movie_inverted_index_compressed.bin"
        vocabulary_file_path = "../data/movie_vocabulary.csv"
        all_ids = movie_all_ids
        words_df = movie_words_df
    else:
        print("输入错误，请输入 1 或 2。")
        continue  # 重新开始循环

    expression = input("请输入布尔查询表达式：\n")
    # 记录查询起始时间
    start_time = time.time()
    tokens = tokenize(expression)
    postfix_tokens = infix_to_postfix(tokens)
    result_ids = evaluate_postfix(postfix_tokens, compressed_file_path, vocabulary_file_path, all_ids)
    # 计算查询耗时
    elapsed_time = time.time() - start_time
     
    if result_ids:
        print("查询结果：\n")
        display_results(result_ids, words_df)
    else:
        print("没有符合条件的结果。")
    
    print("查询耗时：{:.6f} 秒".format(time.time() - start_time))

    cont = input("是否继续查询？(Y/N): ")
    if cont.strip().lower() != 'y':
        print("感谢您的使用！")
        break  # 退出循环，结束程序
